In [ ]:
import numpy as np
import pandas as pd

# 1) 生成 dummy data
rng = np.random.default_rng(42)

n_ids = 500
ids = np.arange(1, n_ids + 1)
refresh_orders = np.arange(0, 10)   # 0=最新，1=前一次，依此类推

# 笛卡尔积：id × refresh_order
df = pd.MultiIndex.from_product([ids, refresh_orders], names=["id", "refresh_order"]).to_frame(index=False)

# 每个 id 给一个基准风险值（1~6），在不同 refresh_order 上加一点扰动
base_risk = rng.integers(1, 7, size=n_ids)
risk_noise = rng.integers(-1, 2, size=(n_ids, len(refresh_orders)))  # -1, 0, 1
risk = np.clip(base_risk[:, None] + risk_noise, 1, 6)

df["risk_num"] = risk.reshape(-1, order="C")

# 2) 生成 MoverPlus 表
lags = {
    "weekly": 1,
    "biweekly": 2,
    "monthly": 4,
    "quarterly": 13,   # dummy 数据里没有13，所以会是 NaN
}

# 只保留 refresh_order=0 的最新行
current = df.loc[df["refresh_order"] == 0, ["id", "risk_num"]].rename(columns={"risk_num": "risk_now"})
moverplus = current.copy()

# 拼接历史数据
for label, lag in lags.items():
    snap = df.loc[df["refresh_order"] == lag, ["id", "risk_num"]].rename(columns={"risk_num": f"risk_{label}_ago"})
    moverplus = moverplus.merge(snap, on="id", how="left")

# 计算差值 + movement 标签
for label in lags.keys():
    moverplus[f"delta_{label}"] = moverplus["risk_now"] - moverplus[f"risk_{label}_ago"]
    def movement(x):
        if pd.isna(x):
            return np.nan
        if x > 0: return "Improved"
        if x < 0: return "Worsened"
        return "No change"
    moverplus[f"movement_{label}"] = moverplus[f"delta_{label}"].map(movement)

# 整理列顺序
order = ["id", "risk_now"]
for label in ["weekly", "biweekly", "monthly", "quarterly"]:
    order += [f"risk_{label}_ago", f"delta_{label}", f"movement_{label}"]
moverplus = moverplus[order].sort_values("id").reset_index(drop=True)

# 查看前几行
print(moverplus.head(20))

In [1]:
import numpy as np
import pandas as pd

# 1) 生成 dummy data
rng = np.random.default_rng(42)

n_ids = 500
ids = np.arange(1, n_ids + 1)
refresh_orders = np.arange(0, 10)   # 0=最新，1=前一次，依此类推

# 笛卡尔积：id × refresh_order
df = pd.MultiIndex.from_product([ids, refresh_orders], names=["id", "refresh_order"]).to_frame(index=False)

# 每个 id 给一个基准风险值（1~6），在不同 refresh_order 上加一点扰动
base_risk = rng.integers(1, 7, size=n_ids)
risk_noise = rng.integers(-1, 2, size=(n_ids, len(refresh_orders)))  # -1, 0, 1
risk = np.clip(base_risk[:, None] + risk_noise, 1, 6)

df["risk_num"] = risk.reshape(-1, order="C")

In [2]:
df

,id,refresh_order,risk_num
0,1,0,1
1,1,1,1
2,1,2,2
3,1,3,2
4,1,4,1
...,...,...,...
4995,500,5,2
4996,500,6,4
4997,500,7,3
4998,500,8,4


In [3]:
# 2) 生成 MoverPlus 表
lags = {
    "weekly": 1,
    "biweekly": 2,
    "monthly": 4,
    "quarterly": 13,   # dummy 数据里没有13，所以会是 NaN
}

# 只保留 refresh_order=0 的最新行
current = df.loc[df["refresh_order"] == 0, ["id", "risk_num"]].rename(columns={"risk_num": "risk_now"})
moverplus = current.copy()

# 拼接历史数据
for label, lag in lags.items():
    snap = df.loc[df["refresh_order"] == lag, ["id", "risk_num"]].rename(columns={"risk_num": f"risk_{label}_ago"})
    moverplus = moverplus.merge(snap, on="id", how="left")

# 计算差值 + movement 标签
for label in lags.keys():
    moverplus[f"delta_{label}"] = moverplus["risk_now"] - moverplus[f"risk_{label}_ago"]
    def movement(x):
        if pd.isna(x):
            return np.nan
        if x > 0: return "Improved"
        if x < 0: return "Worsened"
        return "No change"
    moverplus[f"movement_{label}"] = moverplus[f"delta_{label}"].map(movement)

# 整理列顺序
order = ["id", "risk_now"]
for label in ["weekly", "biweekly", "monthly", "quarterly"]:
    order += [f"risk_{label}_ago", f"delta_{label}", f"movement_{label}"]
moverplus = moverplus[order].sort_values("id").reset_index(drop=True)

In [4]:
moverplus

,id,risk_now,risk_weekly_ago,delta_weekly,movement_weekly,risk_biweekly_ago,delta_biweekly,movement_biweekly,risk_monthly_ago,delta_monthly,movement_monthly,risk_quarterly_ago,delta_quarterly,movement_quarterly
0,1,1,1,0,No change,2,-1,Worsened,1,0,No change,NaN,NaN,NaN
1,2,6,6,0,No change,6,0,No change,6,0,No change,NaN,NaN,NaN
2,3,4,3,1,Improved,3,1,Improved,3,1,Improved,NaN,NaN,NaN
3,4,3,2,1,Improved,2,1,Improved,3,0,No change,NaN,NaN,NaN
4,5,4,2,2,Improved,2,2,Improved,4,0,No change,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,2,4,-2,Worsened,4,-2,Worsened,2,0,No change,NaN,NaN,NaN
496,497,2,1,1,Improved,2,0,No change,1,1,Improved,NaN,NaN,NaN
497,498,3,3,0,No change,4,-1,Worsened,2,1,Improved,NaN,NaN,NaN
498,499,6,5,1,Improved,5,1,Improved,6,0,No change,NaN,NaN,NaN
